<a href="https://colab.research.google.com/github/Danddt/Russian_Opinion_Polls/blob/main/cleaning_data_for_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

UNEMPLOYMENT EXPECTATION INDEX:
"The index is calculated as the difference between the share of those who answered “there will be less unemployed in the next 12 months” and “there will be more unemployed in the next 12 months”. As the index rises, the fear of unemployment decreases."(https://www.levada.ru/indikatory/sotsialno-ekonomicheskie-indikatory/)

# Cleaning Levada files

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.levada.ru/en/ratings/'
response= requests.get(url).content
df_list = pd.read_html(response)


In [ ]:
soup_levada = BeautifulSoup(response, "html.parser")


In [ ]:
#WORK FROM HERE : add empty row to desc
import re

names = soup_levada.find_all('h4')

data_only = df_list[1::2]
descript = df_list[0::2]

d={}
names = []
for header in soup_levada.find_all('h4'):
  test = header.get_text()
  names.append(test)

for item in range(len(names)):
    temp = descript[item]

    temp0 =  pd.DataFrame([['date'] * len(temp .columns)], columns=temp.columns)
    temp1 = temp0.append(temp, ignore_index=True)
    col_name = temp1[0].to_list()
    data = data_only[item].T
    data[0] = data[0].apply(lambda x: "{:.4f}".format(x))
    #data[0] = (data[0].astype(str).str.ljust(6, fillchar='0')) # add '0' at the end of first item in list after split to
    data[0]=(data[0].str.split('.')).apply(lambda x: list(reversed(x))) #reverse to get Year Month (currently Month Year)
    data[0]=data[0].apply('-'.join)
 

    data.columns = col_name
    d[names[item]] = data


In [ ]:
import pathlib
from datetime import date

date = str(date.today())


new_dir = pathlib.Path('/content/drive/MyDrive/RUSSdata/Levada/', date)
new_dir.mkdir(parents=True, exist_ok=True)


In [ ]:
#CREATE FOLDER FOR LEVADA

for data in d:
  d[data].to_csv(str(new_dir)+ '/Levada_' + data+'.csv', index=False)


In [ ]:
# Test read df
test = pd.read_csv('/content/drive/MyDrive/RUSSdata/Levada/2022-09-21/Levada_Putin’s approval rating.csv')
test.tail()

,date,Approve,Disapprove,No answer
270,2022-4,82.0,17.0,1.0
271,2022-5,83.0,15.0,2.0
272,2022-6,83.0,16.0,1.0
273,2022-7,83.0,15.0,2.0
274,2022-8,83.0,15.0,2.0


#Cleaning conflict

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:

link = "https://en.wikipedia.org/wiki/List_of_wars_involving_Russia"
tables = pd.read_html(link)
data= tables[14].iloc[1:]

In [ ]:
data[['start', 'end']]=data['Date'].str.split(pat='–',n=-1, expand=True)
data_reorder = data[['start', 'end', 'Conflict', 'Location', 'Russia (and its allies)','Opponent(s)', 'Result', ]]
data_reorder.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,start,end,Conflict,Location,Russia (and its allies),Opponent(s),Result
1,1991,1992,South Ossetian War,South Ossetia,Georgia,South Ossetia Russia,Victory South Ossetia gained de facto independ...
2,1992,1993,War in Abkhazia,Abkhazia,Abkhazia Russia Gudauta military base Kuban ...,Georgia UNA-UNSO,Victory Abkhazia gained de facto independence
3,1992,None,Transnistria War PMR trucks on the bridge bet...,Transnistria,Transnistria Russia* 14th Guards Army (element...,Moldova Romanian volunteers and military advis...,Victory Transnistria gained de facto independence
4,1992,None,East Prigorodny Conflict,North Ossetia-Alania,Russia North Ossetia-Alania[18],Ingush militia,Victory Expulsion of ethnic Ingush from Prigor...
5,1992,1997,Tajikistani Civil War Spetsnaz troops dismoun...,Tajikistan,/ Tajikistan/ Russia Uzbekistan/ Kazakhstan/ K...,United Tajik Opposition Islamic Renaissance P...,Victory United Nations-sponsored armistice


Select conflicts started or ended after 2004

Additonal data: 

https://en.wikipedia.org/wiki/Second_Chechen_War start : August 1999 / end: April (2009)

https://en.wikipedia.org/wiki/Russo-Georgian_War start:August 2008 / end: August (2008) - N.B.> end of active fighting, the war is frozen

https://en.wikipedia.org/wiki/Insurgency_in_the_North_Caucasus start: April 2009 /end: December (2017)

https://en.wikipedia.org/wiki/Russo-Ukrainian_War start: February 2014 / ongoing

https://en.wikipedia.org/wiki/Russian_military_intervention_in_the_Syrian_civil_war start: September 2015 / ongoing

https://en.wikipedia.org/wiki/Central_African_Republic_Civil_War start(of Russian Intervention): August

https://en.wikipedia.org/wiki/2022_Russian_invasion_of_Ukraine start: February 2022



In [ ]:
conflict_inc=data_reorder[['start', 'end', 'Location']]
short_conf = conflict_inc.iloc[7:, :]

#split "special operation" 1 and 2
ukraine_data= {'start': 2022, 'end': 'ongoing','Location': 'Ukraine'}
short_conf = short_conf.append(ukraine_data, ignore_index=True)

In [ ]:
month_start = [8,8,4,2,9,8,2]
month_end = [4,8,12,'ongoing','ongoing','ongoing', 'ongoing']

In [ ]:
short_conf['month_start'] = month_start
short_conf['month_end']=month_end

In [ ]:
short_conf

,start,end,Location,month_start,month_end
0,1999,2009,Chechnya,8,4
1,2008,None,"Georgia, South Ossetia and Abkhazia",8,8
2,2009,2017,North Caucasus,4,12
3,2014,present,Ukraine (incl. Crimea),2,ongoing
4,2015,present,Syria,9,ongoing
5,2018,present,Central African Republic,8,ongoing
6,2022,ongoing,Ukraine,2,ongoing


In [ ]:
datetime= short_conf['start'].astype(str)+'-'+short_conf['month_start'].astype(str)

In [ ]:
short_conf.insert(loc=0, column='datetime', value=datetime)

In [ ]:
# setting arbitrary value for start of a war to 100
short_conf['arb_val_war']= 100

In [ ]:
import pathlib
from datetime import date

date = str(date.today())


new_dir = pathlib.Path('/content/drive/MyDrive/RUSSdata/wiki/', date)
new_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
short_conf.to_csv('/content/drive/MyDrive/RUSSdata/wiki/'+date+'/conflicts_russian_federation.csv', index=False) #use this when running whole notebook
data_reorder.to_csv('/content/drive/MyDrive/RUSSdata/wiki/'+date+'/conflicts_russian_federation_long.csv', index=False) 


# Cleaning Terror incidents

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/Terrorism_in_Russia'

page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
import calendar
months = (list(calendar.month_name))
months.pop(0)
month_pattern =('|'.join(months))

In [ ]:
import re
temp=[]

for header in soup.find_all(['h4']):
  test = header.get_text()
  test= str(test)
  test = test[:-6]
  

  for elem in header.next_siblings:
    if elem.name and elem.name.startswith('h'):
        break
    if elem.name == 'p':
      ev = elem.get_text()
      ev= ev.strip()
      ev = re.sub(r'\[.*?\]', "",ev )
      ev=str(ev)
      for month in months:
        if month in ev:
          mo = month
      
      data = test, mo, ev
      temp.append(data)

In [ ]:
import pandas as pd

df = pd.DataFrame.from_records(temp, columns=['year','month',  'event'])
df1 = df[df['event'].str.contains(month_pattern)]


In [ ]:
#Retrieve city names
import pandas as pd

link = "https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia_by_population"
tables = pd.read_html(link)
data = tables[0]

In [ ]:
cities= data['City / Town']

In [ ]:
city_list = cities['City / Town'].to_list()
city_list.append('Beslan')
city_list.append('St Petersburg')
city_list.remove('Vladimir')
city_list.append('Achkhoy-Martonovsky')

In [ ]:
df1['city_name_mapped'] = (df1.event.str.findall('|'.join(city_list)).str[:2]) #.apply(', '.join)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df1['city_name_mapped'].str[0]==df1['city_name_mapped'].str[1]

for item in df1['city_name_mapped']:
  if len(item)>1:
    if item[0] in item[1]:
      item.pop()
      

In [ ]:
df1['city_name_mapped'] = (df1.city_name_mapped.apply(', '.join))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
import numpy as np
df1['city_name_mapped'].replace('', np.nan, inplace=True)
df1.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
listo = []
for header in soup.find_all(['h3']):
  for elem in header.next_siblings:
    if elem.name and elem.name.startswith('h'):
      break
    if elem.name == 'p':
        txt = str(elem.get_text())
        txt = re.sub("\[.*?\]","",txt)
        splito = txt.split('\n')
        listo.append(splito)

In [ ]:
short_list = listo[-6:]

In [ ]:
flat_list = [item for sublist in short_list[1::2] for item in sublist]

In [ ]:
month_date=[]
for item in flat_list:
  for mo in months:
    if mo in item:
      month_date.append(mo)

In [ ]:
from_2002 = pd.DataFrame({'year':(short_list[::2]), 'month':month_date , 'event':(short_list[1::2]),})
from_2002['year'] = (from_2002['year'].apply(', '.join)).str.replace(',','')
from_2002['event'] = from_2002['event'].apply(', '.join)
from_2002['city_name_mapped'] = (from_2002.event.str.findall('|'.join(city_list)).str[:2]).apply(', '.join)

In [ ]:
terror_incidents = pd.concat([from_2002, df1])

In [ ]:
datetime_t = terror_incidents.year.str.strip() +"-"+ pd.to_datetime(terror_incidents.month, format='%B').dt.month.astype(str) 

In [ ]:
terror_incidents.insert(loc=0, column='datetime', value=datetime_t)

In [ ]:
# setting arbitrary value for terror incident at 80
terror_incidents['arb_val_terror']= 80

In [ ]:
terror_incidents.to_csv('/content/drive/MyDrive/RUSSdata/wiki/'+date+'/terror_incidents.csv', index=False) #use this when running whole notebook


# Cleaning censorship, application of the law, protests,  files

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
url1 = "https://en.wikipedia.org/wiki/Internet_censorship_in_Russia"
page = requests.get(url1)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
import calendar
months = (list(calendar.month_name))
months.pop(0)
month_pattern =('|'.join(months))

In [ ]:
yer = []
for item in range(2000,2023 ):
  yer.append(str(item))

pattern = ('|'.join(yer))


In [ ]:
import re
temp=[]

for header in soup.find_all(['h4']):
  test = header.get_text()
  test= str(test)
  test = test[:-6]
  

  for elem in header.next_siblings:
    if elem.name and elem.name.startswith('h'):
        break
    if elem.name == 'p':

      ev = elem.get_text()
      # ev= ev.strip()
      ev = re.sub(r'\[.*?\]', "",ev )
      ev=str(ev)

      for month in months:
        if month in ev:
          mo = month
      for year in yer:
        if year in ev:
          yyear= year
  
      
      data = yyear,mo, ev
      temp.append(data)

In [ ]:
import pandas as pd

soupp1_test = pd.DataFrame.from_records(temp, columns=['year', 'month', 'event'])

In [ ]:
soup1_group = soupp1_test.groupby(['year','month'])['event'].apply(' \\'.join).reset_index()
count = (soupp1_test.groupby(['year','month']).count()).reset_index()
soup1_group['event_occurances']  =count['event']

In [ ]:
url2 = "https://en.wikipedia.org/wiki/List_of_websites_blocked_in_Russia"
page2 = requests.get(url2)
soup2 = BeautifulSoup(page2.content, 'html.parser')

In [ ]:
import re


sentence_list = []
year_list= []
prefixes = ('^','1','2','3','4')
len_num = len(soup2.find_all('div'))

for item in range(0, len_num):
  temp =  soup2.find_all('li')[item].get_text()
  temp = temp.strip()
  temp = re.sub(r'\[.*?\]', "",temp )
  temp=str(temp)
  if not temp.startswith(prefixes):

    for month in months:
      if month in temp:
        mo = month
    for year in yer:
      if year in temp:
        yyear= year

    data1 = yyear, mo, temp
    sentence_list.append(data1)

In [ ]:
import pandas as pd

soupp2_test = pd.DataFrame.from_records(sentence_list[:-3], columns=['year', 'month', 'event'])


In [ ]:
soup2_group = soupp2_test.groupby(['year','month'])['event'].apply(' \\'.join).reset_index()
count2 = (soupp2_test.groupby(['year','month']).count()).reset_index()
soup2_group['event_occurances']  =count2['event']

In [ ]:
url3 = 'https://en.wikipedia.org/wiki/Media_freedom_in_Russia'

page3 = requests.get(url3)
soup3 = BeautifulSoup(page3.content, 'html.parser')

In [ ]:
import re
temp3=[]


for ele in soup3.find_all('p'):
  test3 = ele.get_text()
  test3 = re.sub(r'\[.*?\]' , "",test3 )
  test3= str(test3)

  for month in months:
    if month in test3:
      mo = month
  for year in yer:
    if year in test3:
      yyear= year
  

  
  data3 = yyear, mo, test3
  temp3.append(data3)


In [ ]:
import pandas as pd

soupp3_test = pd.DataFrame.from_records(temp3, columns=['year', 'month', 'event'])
# drop empty event row
#soupp3_test[soupp3_test['event'].map(len)<6]
#>>> 	  year 	month 	event
#>>>0 	2015 	October 	\n
soupp3_test.drop(0, inplace=True)

In [ ]:
soupp3_group = soupp3_test.groupby(['year','month'])['event'].apply(' \\'.join).reset_index()
count3 = (soupp3_test.groupby(['year','month']).count()).reset_index()
soupp3_group ['event_occurances'] = count3['event']

In [ ]:
censor1_2_3 =  pd.concat([soup1_group, soup2_group, soupp3_group])

In [ ]:
censor1_2_3group = censor1_2_3.groupby(['year','month'])['event'].apply(' \\'.join).reset_index()
count_all= (censor1_2_3.groupby(['year','month'])['event_occurances'].sum()).reset_index()
censor1_2_3group['event_occurances'] = count_all['event_occurances']

In [ ]:
datetime_c = censor1_2_3group.year.str.strip() +"-"+ pd.to_datetime(censor1_2_3group.month, format='%B').dt.month.astype(str) 

In [ ]:
censor1_2_3group.insert(loc=0, column='datetime', value=datetime_c)

**Adding in recent data : additional**

*getting tricky to keep on top of this...*

Wikipedia:

"https://en.wikipedia.org/wiki/2022_anti-war_protests_in_Russia"


Additional information alternative saurces (not wikipedia)

Kara-Murza : Russian Opposition Activist Kara-Murza Sentenced To 15 Days In Jail source: https://www.rferl.org/a/russia-kara-murza-arrested/31798286.html

Ilya Yashin : arrest https://www.themoscowtimes.com/2022/07/13/russia-arrests-anti-war-opposition-activist-ilya-yashin-a78288

Leonid Gozman :arrest source: https://apnews.com/article/russia-ukraine-arrests-moscow-4cf9bb89d486fccb4ad2ff7299b28b1f

Andrei Pivovarov : Russian opposition activist Andrei Pivovarov jailed for 4 years source: https://www.aljazeera.com/news/2022/7/15/russian-opposition-activist-andrei-pivovarov-jailed-for-4-years

Helga Pirogova : Siberian Opposition Deputy Flees Criminal 'War Fakes' Prosecution source : https://www.themoscowtimes.com/2022/07/25/siberian-opposition-deputy-flees-criminal-war-fakes-prosecution-a78395

Yevgeny Roizman : https://www.aljazeera.com/news/2022/8/24/russia-arrests-ex-mayor-for-discrediting-army-amid-ukraine-war



In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
url4 = "https://en.wikipedia.org/wiki/2022_anti-war_protests_in_Russia"
page4 = requests.get(url4)
soup4 = BeautifulSoup(page.content, 'html.parser')

In [ ]:
import calendar
months = (list(calendar.month_name))
months.pop(0)
month_pattern =('|'.join(months))

In [ ]:
import re
temp4=[]

for header in soup4.find_all(['h3']):
  test = header.get_text()
  test= str(test)

  
  for elem in header.next_siblings:
    if elem.name and elem.name.startswith('h'):
        break
    if elem.name == 'p':
      ev = elem.get_text()
      ev= ev.strip()
      ev = re.sub(r'\[.*?\]', "",ev )
      ev=str(ev)
      for month in months:
        if month in ev:
          mo = month
      
      data = mo, ev
      temp4.append(data)


In [ ]:
add_on =pd.DataFrame.from_records(temp4, columns=['month', 'event'])

In [ ]:
add_on_s = add_on[add_on['event'].str.contains('2022', regex=True)]

In [ ]:
add_on_s['datetime'] = '2022'
add_on_s['datetime'] =add_on_s.datetime+'-'+pd.to_datetime(add_on_s.month, format='%B').dt.month.astype(str) 
add_on_s = add_on_s[1:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
add_on_s

,month,event,datetime
24,March,"On 11 March 2022, Belarusian political police ...",2022-3
25,July,"In April-July 2022, the Russian authorities pu...",2022-7
26,February,Russian authorities have blocked or removed ab...,2022-2


In [ ]:
event=['Kara-Murza : Russian Opposition Activist Kara-Murza Sentenced To 15 Days In Jail',
       'Ilya Yashin : arrested', 'Leonid Gozman :arrested',
       'Andrei Pivovarov : Russian opposition activist Andrei Pivovarov jailed for 4 years',
       'Helga Pirogova : Siberian Opposition Deputy Flees Criminal \'War Fakes\' Prosecution ' ]

In [ ]:
supp_july = pd.DataFrame(event, columns = ['event'])
supp_july['datetime']= '2022-7'

In [ ]:
supp_july

,event,datetime
0,Kara-Murza : Russian Opposition Activist Kara-...,2022-7
1,Ilya Yashin : arrested,2022-7
2,Leonid Gozman :arrested,2022-7
3,Andrei Pivovarov : Russian opposition activist...,2022-7
4,Helga Pirogova : Siberian Opposition Deputy Fl...,2022-7


In [ ]:
event_aug = ['Yevgeny Roizman: Russia arrests ex-mayor for discrediting army amid Ukraine war']
supp_aug = pd.DataFrame(event_aug, columns = ['event'])
supp_aug['datetime']= '2022-8'

In [ ]:
selected_add = add_on_s.append(supp_july, ignore_index=True)
selected_add =selected_add.append(supp_aug, ignore_index=True)

In [ ]:
selected_add

,month,event,datetime
0,March,"On 11 March 2022, Belarusian political police ...",2022-3
1,July,"In April-July 2022, the Russian authorities pu...",2022-7
2,February,Russian authorities have blocked or removed ab...,2022-2
3,NaN,Kara-Murza : Russian Opposition Activist Kara-...,2022-7
4,NaN,Ilya Yashin : arrested,2022-7
5,NaN,Leonid Gozman :arrested,2022-7
6,NaN,Andrei Pivovarov : Russian opposition activist...,2022-7
7,NaN,Helga Pirogova : Siberian Opposition Deputy Fl...,2022-7
8,NaN,Yevgeny Roizman: Russia arrests ex-mayor for d...,2022-8


In [ ]:
count_all= (selected_add.groupby(['datetime'])['event'].count()).to_list()

In [ ]:
selected_g = (selected_add.groupby(['datetime'])['event'].sum()).reset_index()
selected_g['event_occurances'] = count_all

In [ ]:
selected_g

,datetime,event,event_occurances
0,2022-2,Russian authorities have blocked or removed ab...,1
1,2022-3,"On 11 March 2022, Belarusian political police ...",1
2,2022-7,"In April-July 2022, the Russian authorities pu...",6
3,2022-8,Yevgeny Roizman: Russia arrests ex-mayor for d...,1


In [ ]:
events_all = pd.concat([censor1_2_3group, selected_g])
events_all.drop(columns=['year', 'month'], inplace=True)

Arbitrary score for culmulative effect legislation and active policing: 
Value assignment is based on the following saying:

“Строгость российских законов смягчается необязательностью их исполнения(Ru.m.wikipedia.org) 

“The severity of Russian laws is alleviated by the lack of obligation to fulfill them” (Translation kindly provided by SetarkoYT, https://twitter.com/SetarkoYT), 

Therefore by extension, although the passing of a new law or amendment may be noted (score=1), it will have little impact, until a very visible implementation, in the form of, for example, a jail sentence, occurs (score=2).

In [ ]:
import numpy as np
key_words =['filtering', 'implementation', 'blacklist', 'block', 'law', 'restricted', 'ban', 'isolate', 
            'fine', 'harrasment', 'criminal','throttling', 'step down', 'resign', 'killings', 'harassed', 'suppress', 'detained', 'arrested']
key_words2 = ['jail', 'sentenced', 'prison']

key_words_regex = "|".join(key_words)
key_words2_regex = "|".join(key_words2)

In [ ]:
events_all['arbitrary_score'] = np.add((np.where(events_all['event'].str.contains(key_words_regex, regex=True), 1, 0.1)), 
       (np.where(events_all['event'].str.contains(key_words2_regex, regex=True), 2, 0.1)))

In [ ]:

events_all['count_words']=events_all['event'].str.count(key_words_regex)
events_all['count_word2']=events_all['event'].str.count(key_words2_regex)
events_all.reset_index(drop=True, inplace=True)

In [ ]:
arb_score = ((events_all['count_words'] + events_all['count_word2']+events_all['event_occurances'] ) *events_all['arbitrary_score']).cumsum()

In [ ]:
a, b = 1, 95
x, y = arb_score.min(), arb_score.max()
cens_score = (arb_score - x) / (y - x) * (b - a) + a
cens_score.tail()

65    89.499602
66    89.828431
67    90.486087
68    94.970107
69    95.000000
dtype: float64

In [ ]:
events_all['censor_score'] = cens_score
events_all.tail()

,datetime,event,event_occurances,arbitrary_score,count_words,count_word2,censor_score
65,2022-3,"In March 2022, amid its invasion of Ukraine, R...",14,3.0,16,1,89.499602
66,2022-2,Russian authorities have blocked or removed ab...,1,1.1,1,0,89.828431
67,2022-3,"On 11 March 2022, Belarusian political police ...",1,1.1,3,0,90.486087
68,2022-7,"In April-July 2022, the Russian authorities pu...",6,3.0,3,1,94.970107
69,2022-8,Yevgeny Roizman: Russia arrests ex-mayor for d...,1,0.2,0,0,95.000000


In [ ]:
events_all.to_csv('/content/drive/MyDrive/RUSSdata/wiki/'+date+'/censorship_plus.csv', index=False)#use this when running whole notebook


# Cleaning journalists data

In [ ]:
url_j = 'https://en.wikipedia.org/wiki/List_of_journalists_killed_in_Russia'

page_j = requests.get(url_j)
soup_j = BeautifulSoup(page_j.content, 'html.parser')

In [ ]:
testo1 = []
for header in soup_j.find_all(['h4']):
  head = (header.get_text())
  head = re.sub(r'\[.*?\]', "",head )
  for ele in header.find_next_sibling():
    if ele.name == 'li':
      event=(ele.get_text())
      event=re.sub(r'\[.*?\]', "",event)
      for month in months:
        if month in event:
          mo =month
      datum = head,mo, event
      testo1.append(datum)

In [ ]:
journalist = pd.DataFrame.from_records(testo1, columns=['year', 'month','event'])

In [ ]:
journalist_group = journalist.groupby(['year','month'])['event'].apply(' \\'.join).reset_index()
count_j = (journalist.groupby(['year','month']).count()).reset_index()
journalist_group['event_occurances'] = count_j ['event']

In [ ]:
tables = pd.read_html(url_j)

In [ ]:
tablej = tables[0]

In [ ]:
tablej = tablej .T.reset_index()

In [ ]:
cols = tablej.iloc[0]

In [ ]:
tablej_head = tablej[1:-1]

In [ ]:
tablej_head.columns= cols

In [ ]:
wiki_tb = tablej_head[['Year', 'Murder only']]

In [ ]:
wiki_tb.columns = ['year', 'event_occurances']

In [ ]:
date_time_j = journalist_group.year.str.strip() +"-"+ pd.to_datetime(journalist_group.month, format='%B').dt.month.astype(str) 
journalist_group.insert(loc=0, column='datetime', value=date_time_j )

In [ ]:
journl_sft = journalist_group.groupby('year')['event_occurances'].sum().reset_index()

In [ ]:
add_on_j = journl_sft[2:]

In [ ]:
journalists_death = pd.concat([wiki_tb, add_on_j])
journalists_death.tail()

,year,event_occurances
6,2014,1
7,2016,1
8,2017,4
9,2018,5
10,2022,3


In [ ]:
journalists_death.to_csv('/content/drive/MyDrive/RUSSdata/wiki/'+date+'/journalists_death.csv', index=False)#use this when running whole notebook
#journalist_group.to_csv('/content/drive/MyDrive/RUSSdata/2022-07-20/journalists_death.csv', index=False)

# Fred Data

In [ ]:
!pip install full-fred

# https://github.com/7astro7/full_fred

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 4.7 MB/s 


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from full_fred.fred import Fred

fred = Fred('/content/drive/MyDrive/RUSSdata/fred_api_key.txt')

fred.get_api_key_file()

'/content/drive/MyDrive/RUSSdata/fred_api_key.txt'

In [ ]:
commodities = ['WPU01830161','PSUNOUSDM','PWHEAMTUSDM', 'DCOILBRENTEU', 'PNGASEUUSDM','POILBREUSDM', 'CCUSMA02RUM618N', 'SIPOVGINICHN','SIPOVGINIRUS','SIPOVGINIUSA','NYGDPPCAPKDCHN','NYGDPPCAPKDRUS','NYGDPPCAPKDUSA']
names = ['sunflower','sunflower_oil', 'wheat', 'oil_eu', 'natural_gas_eu', 'Brent_Crude_Global','ruble', 'GINI_China','GINI_Rus','GINI_US','Constant_GDP_per_capita_China','Constant_GDP_per_capita_Rus','Constant_GDP_per_capita_US' ]


Citations:

U.S. Bureau of Labor Statistics, Producer Price Index by Commodity: Farm Products: Sunflower [WPU01830161], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/WPU01830161, July 21, 2022.

International Monetary Fund, Global price of Sunflower Oil [PSUNOUSDM], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/PSUNOUSDM, July 20, 2022.

International Monetary Fund, Global price of Wheat [PWHEAMTUSDM], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/PWHEAMTUSDM, July 20, 2022.

U.S. Energy Information Administration, Crude Oil Prices: Brent - Europe [DCOILBRENTEU], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/DCOILBRENTEU, July 20, 2022.

International Monetary Fund, Global price of Natural gas, EU [PNGASEUUSDM], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/PNGASEUUSDM, July 20, 2022.

International Monetary Fund, Global price of Brent Crude [POILBREUSDM], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/POILBREUSDM, July 20, 2022.

Organization for Economic Co-operation and Development, National Currency to US Dollar Exchange Rate: Average of Daily Rates for the Russian Federation [CCUSMA02RUM618N], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/CCUSMA02RUM618N, July 20, 2022


World Bank, GINI Index for the United States [SIPOVGINIUSA], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/SIPOVGINIUSA, July 24, 2022.

https://fred.stlouisfed.org/series/NYGDPPCAPKDCHN World Bank, Constant GDP per capita for China [NYGDPPCAPKDCHN], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/NYGDPPCAPKDCHN, July 23, 2022.

https://fred.stlouisfed.org/series/NYGDPPCAPKDRUS World Bank, Constant GDP per capita for the Russian Federation [NYGDPPCAPKDRUS], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/NYGDPPCAPKDRUS, July 23, 2022.

https://fred.stlouisfed.org/series/NYGDPPCAPKDUSA

World Bank, Constant GDP per capita for the United States [NYGDPPCAPKDUSA], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/NYGDPPCAPKDUSA, July 23, 2022. 

https://www.reuters.com/world/china/what-is-chinas-common-prosperity-drive-why-does-it-matter-2021-09-02/ BEIJING, Sept 2 (Reuters) - President Xi Jinping has called for China to achieve "common prosperity", seeking to narrow a yawning wealth gap that threatens the country's economic ascent and the legitimacy of Communist Party rule.

President Xi Assumed office 15 November 2012

In [ ]:
d_names = dict(zip(commodities,names))


In [ ]:
comm_data= {}

for item in commodities:
  temp= fred.get_series_df(item)
  temp = temp[['date', 'value']]
  temp['value'] = pd.to_numeric(temp['value'], errors='coerce').fillna(np.NaN)
  temp = temp.rename(columns={'value':item})
  temp['date'] = pd.to_datetime(temp['date'],format='%Y/%m/%d')
  #df_list.append(temp)
  comm_data[item] = temp


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Change from 1 Dollar to Ruble ---> 1 Ruble to Dollar

comm_data['CCUSMA02RUM618N']['CCUSMA02RUM618N'] = 1/comm_data['CCUSMA02RUM618N']['CCUSMA02RUM618N']


In [ ]:
import pathlib
from datetime import date

date = str(date.today())


new_dir = pathlib.Path('/content/drive/MyDrive/RUSSdata/Fred_data/', date)
new_dir.mkdir(parents=True, exist_ok=True)

for data in comm_data:
  comm_data[data].to_csv(str(new_dir)+'/'+ data+'.csv', index=False)

# World Bank data

In [ ]:
!pip install wbgapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wbgapi as wb
import pandas as pd
import numpy as np

In [ ]:
# source = https://data.worldbank.org
#NY.GDP.PCAP.CD?locations=RU gdpp per capita Russia
#NY.GDP.MKTP. GDP values
#NY.GDP.PCAP.CD values GDPP

code= ['VC.IHR.PSRC.P5', 'SH.ALC.PCAP.LI', 'SI.POV.NAHC','SI.POV.GINI', 'SH.XPD.GHED.PP.CD']
name = ['Intentional homicides (per 100,000 people)', 'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)', 'Poverty headcount ratio at national poverty lines (% of population)',
        'Gini index','Domestic general government health expenditure per capita, PPP (current international $)',]

data = wb.data.DataFrame(['VC.IHR.PSRC.P5', 'SH.ALC.PCAP.LI', 'SI.POV.NAHC', 'SI.POV.GINI','SH.XPD.GHED.PP.CD',], 'RUS', mrv=31) # get 31 years # Vladimir Putin first as Prime Minister under Yeltsin then president 1999

In [ ]:
data_wb = data.T.reset_index()
data_wb['index']= data_wb['index'].str.replace('YR', '')
data_wb.head()

series,index,SH.ALC.PCAP.LI,SH.XPD.GHED.PP.CD,SI.POV.GINI,SI.POV.NAHC,VC.IHR.PSRC.P5
0,1990,NaN,NaN,NaN,NaN,14.332526
1,1991,NaN,NaN,NaN,NaN,15.284916
2,1992,NaN,NaN,NaN,NaN,22.872008
3,1993,NaN,NaN,NaN,NaN,30.369287
4,1994,NaN,NaN,NaN,NaN,32.268406


In [ ]:
values_GDPP = wb.data.DataFrame(['NY.GDP.PCAP.CD', ], ['RUS', 'USA'], mrv=31) #gdpp per capita values
values_GDP = wb.data.DataFrame(['NY.GDP.MKTP.CD', ], ['RUS', 'USA'], mrv=31) #gdp values 

In [ ]:
values_GDPP = values_GDPP.T.reset_index()
values_GDPP['index']= values_GDPP['index'].str.replace('YR', '')
values_GDPP.columns=['year', 'RUS_gdpp','USA_gdpp', ]
values_GDPP.head(3)

,year,RUS_gdpp,USA_gdpp
0,1991,3490.452393,24342.258905
1,1992,3098.802734,25418.990776
2,1993,2930.670166,26387.293734


In [ ]:
values_GDP = values_GDP.T.reset_index()
values_GDP['index']= values_GDP['index'].str.replace('YR', '')
values_GDP.columns=['year', 'RUS_gdp','USA_gdp', ]
values_GDP.head(3)

,year,RUS_gdp,USA_gdp
0,1991,5.179630e+11,6.158129e+12
1,1992,4.602906e+11,6.520327e+12
2,1993,4.350837e+11,6.858559e+12


In [ ]:
wb_GDP_grow = wb.data.DataFrame(['NY.GDP.MKTP.KD.ZG'], ['RUS', 'USA'], mrv=26)  # GDP growth (annual %) 
wb_GDPP_grow = wb.data.DataFrame(['NY.GDP.PCAP.KD.ZG'], ['RUS', 'USA'], mrv=26) # GDPP percapita growth (annual %) 

In [ ]:
wb_GDP_grow = wb_GDP_grow.T.reset_index()
wb_GDP_grow['index']= wb_GDP_grow['index'].str.replace('YR', '')
wb_GDP_grow.columns = ['year', 'RUS_gdp', 'USA_gdp']
wb_GDP_grow.head(3)

,year,RUS_gdp,USA_gdp
0,1996,-3.755069,3.772565
1,1997,1.399916,4.447175
2,1998,-5.299962,4.481394


In [ ]:
wb_GDPP_grow = wb_GDPP_grow.T.reset_index()
wb_GDPP_grow['index']= wb_GDPP_grow['index'].str.replace('YR', '')
wb_GDPP_grow.columns = ['year', 'RUS_gdPP', 'USA_gdPP']
wb_GDPP_grow.head(3)

,year,RUS_gdPP,USA_gdPP
0,1996,-3.614975,2.572259
1,1997,1.567704,3.197212
2,1998,-5.143115,3.270511


In [ ]:
### save both versions 'as is' and normalized:
import pathlib
from datetime import date

date = str(date.today())


new_dir = pathlib.Path('/content/drive/MyDrive/RUSSdata/World_Bank/', date)
new_dir.mkdir(parents=True, exist_ok=True)

data_wb.to_csv(str(new_dir)+'/'+'w_bank_data.csv', index=False)
wb_GDP_grow.to_csv(str(new_dir)+'/'+'w_bank_GDP_grow.csv', index=False)
wb_GDPP_grow.to_csv(str(new_dir)+'/'+'w_bank_GDPP_grow.csv', index=False)
values_GDP.to_csv(str(new_dir)+'/'+'w_bank_GDP_value.csv', index=False)
values_GDPP.to_csv(str(new_dir)+'/'+'w_bank_GDPP_value.csv', index=False)

In [ ]:
cols = ['SH.ALC.PCAP.LI', 'SH.XPD.GHED.PP.CD', 'SI.POV.GINI',
       'SI.POV.NAHC', 'VC.IHR.PSRC.P5']
data_wb[cols] = data_wb[cols].apply(pd.to_numeric, errors='coerce').fillna(np.nan)
data_wb.head()

series,index,SH.ALC.PCAP.LI,SH.XPD.GHED.PP.CD,SI.POV.GINI,SI.POV.NAHC,VC.IHR.PSRC.P5
0,1990,NaN,NaN,NaN,NaN,14.332526
1,1991,NaN,NaN,NaN,NaN,15.284916
2,1992,NaN,NaN,NaN,NaN,22.872008
3,1993,NaN,NaN,NaN,NaN,30.369287
4,1994,NaN,NaN,NaN,NaN,32.268406


In [ ]:
df_norm = (data_wb[cols]-data_wb[cols].min())/ (data_wb[cols].max() - data_wb[cols].min())

In [ ]:
read_name = {'SH.ALC.PCAP.LI': 'alcohol consumption', 
'SH.XPD.GHED.PP.CD': 'Health Expenditure',
'SI.POV.GINI':'Gini Index',
'SI.POV.NAHC': 'Poverty',
'VC.IHR.PSRC.P5':'Homicide',}

In [ ]:
# Normalize data for ease of graphing
df_norm = df_norm.rename(columns =read_name)
df_norm .head()

series,alcohol consumption,Health Expenditure,Gini Index,Poverty,Homicide
0,NaN,NaN,NaN,NaN,0.280793
1,NaN,NaN,NaN,NaN,0.318982
2,NaN,NaN,NaN,NaN,0.623216
3,NaN,NaN,NaN,NaN,0.923848
4,NaN,NaN,NaN,NaN,1.000000


In [ ]:
df_norm.insert(0,'Year' ,data_wb['index'])

In [ ]:
df_norm.head()

series,Year,alcohol consumption,Health Expenditure,Gini Index,Poverty,Homicide
0,1990,NaN,NaN,NaN,NaN,0.280793
1,1991,NaN,NaN,NaN,NaN,0.318982
2,1992,NaN,NaN,NaN,NaN,0.623216
3,1993,NaN,NaN,NaN,NaN,0.923848
4,1994,NaN,NaN,NaN,NaN,1.000000


In [ ]:
short = df_norm[df_norm['Year']>'1998']

In [ ]:
### save both versions 'as is' and normalized:
import pathlib
from datetime import date

date = str(date.today())


new_dir = pathlib.Path('/content/drive/MyDrive/RUSSdata/World_Bank/', date)
new_dir.mkdir(parents=True, exist_ok=True)

short.to_csv(str(new_dir)+'/'+'w_b_normalized.csv', index=False)
df_norm.to_csv(str(new_dir)+'/'+'w_b_normalized_long.csv', index=False)


# Inflation and unemployment data

sources:

'https://www.statbureau.org/ru' - inflation

'https://ruxpert.ru/' - unemploymnet - I cross checked the data on the original source RosStat: https://rosstat.gov.ru/


In [ ]:
import pathlib
from datetime import date

date = str(date.today())


new_dir = pathlib.Path('/content/drive/MyDrive/RUSSdata/Russ_Stat_Office/', date)
new_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
import pandas as pd

link_ru = 'https://www.statbureau.org/ru/russia/inflation-tables'

tables = pd.read_html(link_ru)



In [ ]:
data = tables[1]
year = data['Год']
data_val = data.iloc[:, 1:]/100
data_val.insert(0, 'Year', year, True)

In [ ]:
# renaiming columns to English:
column_names = ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec','Total']

data_val.columns=[column_names]

In [ ]:
data_val.to_csv(str(new_dir)+'/'+'inflation_russia.csv', index=False) 

In [ ]:
import pandas as pd

unempl= 'https://ruxpert.ru/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0:%D0%A3%D1%80%D0%BE%D0%B2%D0%B5%D0%BD%D1%8C_%D0%B1%D0%B5%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B8%D1%86%D1%8B_%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # not working downloaded xlx from https://rosstat.gov.ru/labour_force

unempl_tables = pd.read_html(unempl)

In [ ]:
data = unempl_tables[0]

In [ ]:
data.columns=['Year', 'Unemployment rate(million people)', 'Unemployment rate(% of population)' ]


In [ ]:
import pandas as pd
# typo on original site https://ruxpert.ru/, coma instead of dot

data.at[2,'Unemployment rate(% of population)']=7.9

In [ ]:
data.head(3)

,Year,Unemployment rate(million people),Unemployment rate(% of population)
0,2000,7.700,10.6
1,2001,6.424,9.0
2,2002,5.698,7.9


adding back in :
data for year 2021 and 2022, downloaded xlx from https://rosstat.gov.ru/labour_force

In [ ]:
#add back in 2021
year_2021= {'Year':'2021','Unemployment rate(million people)':3.630, 'Unemployment rate(% of population)':4.8 }

data = data.append(year_2021, ignore_index=True)
data.head()

,Year,Unemployment rate(million people),Unemployment rate(% of population)
0,2000,7.700,10.6
1,2001,6.424,9.0
2,2002,5.698,7.9
3,2003,5.934,8.2
4,2004,5.666,7.8


In [ ]:
unemp_ross= pd.read_csv('/content/drive/MyDrive/RUSSdata/Russ_Stat_Office/unemp_2022_rostat.txt', sep='\t')
unemp_ross_2022 = unemp_ross.iloc[:, :-5]

In [ ]:
unemp_ross_2022 #save this 

,year,Unemployment rate (million people),Unemployment rate (% of population)
0,2022-01,3.296,4.4
1,2022-02,3.071,4.1
2,2022-03,3.078,4.1
3,2022-04,3.005,4.0
4,2022-05,2.946,3.9
5,2022-06,2.951,3.9
6,2022-07,2.900,3.9


In [ ]:
#append average so far
new_row =['2022',unemp_ross_2022['Unemployment rate (million people)'].mean(),unemp_ross_2022['Unemployment rate (% of population)'].mean() ]
year_2022 = {'Year':'2022','Unemployment rate(million people)':unemp_ross_2022['Unemployment rate (million people)'].mean(),
             'Unemployment rate(% of population)':unemp_ross_2022['Unemployment rate (% of population)'].mean()}
data = data.append(year_2022, ignore_index=True)
data.head()

,Year,Unemployment rate(million people),Unemployment rate(% of population)
0,2000,7.700,10.6
1,2001,6.424,9.0
2,2002,5.698,7.9
3,2003,5.934,8.2
4,2004,5.666,7.8


In [ ]:
data.to_csv('/content/drive/MyDrive/RUSSdata/Russ_Stat_Office/yearly_unemployment.csv', index=False)
unemp_ross_2022.to_csv('/content/drive/MyDrive/RUSSdata/Russ_Stat_Office/monthly_unemployment_2022.csv', index=False)
